In [1]:
import sys
sys.path.append("../scr/utils")

import os
import yaml
import time
import datetime
import warnings
import platform
from tqdm import tqdm

import torch
import timm
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

from create_dataset import BirdDataset
from base_utils import set_seed
from metrics import validation_epoch_end

In [2]:
# Информация о железе, на котором тестируется модель

processor_info = platform.processor()
print("Информация о процессоре:", processor_info)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print("Доступно GPU:", num_gpus)
    for i in range(num_gpus):
        gpu = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu}")   
else:
    print("GPU недоступны на данной системе.")

Информация о процессоре: Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
Доступно GPU: 1
GPU 1: NVIDIA GeForce GTX 1070 Ti


In [3]:
df = pd.read_csv("../data/data.csv")
df_test = df[df.fold == 3].reset_index(drop=True)                     

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model_base = torch.load("../experiment/14_September_2023_16_29/model_tf_efficientnet_b0_last_version.pt").to(device)

In [6]:
dataset_test = BirdDataset(df=df_test,
                           path_to_folder_with_audio="../data")

In [7]:
valid_loader = DataLoader(dataset_test,
                          batch_size=4)

In [8]:
model_base.eval()
device

device(type='cuda')

In [9]:
# %%timeit -n 4
## Найдем скорость работы базовой модели на нашем валидационном датасете
predicted_labels_list = None
true_labels_list = None
metric = validation_epoch_end
start_time = time.time()

model = model_base
with torch.no_grad():
    for batch in tqdm(valid_loader):
        X_batch = batch[0].to(device)
        y_batch = batch[1].to(device)
        res = model_base.forward(X_batch)

        res = res.detach().cpu().sigmoid().numpy()
        y_batch_onehot = y_batch
        y_batch_onehot = y_batch_onehot.unsqueeze(1).detach().cpu().numpy()
        y_batch_onehot = y_batch_onehot.squeeze()

        if predicted_labels_list is None:
            predicted_labels_list = res
            true_labels_list = y_batch_onehot
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)      

        del batch, res
    torch.cuda.empty_cache()

end_time = time.time()



        

100%|██████████| 771/771 [03:21<00:00,  3.83it/s]


In [10]:
all_predicted_labels = np.vstack(predicted_labels_list)
all_true_labels = np.vstack(true_labels_list)
all_true_labels = np.squeeze(all_true_labels)
mask = (all_true_labels > 0) & (all_true_labels < 1)
all_true_labels[mask] = 0
avg_metric = metric(all_true_labels, all_predicted_labels)

In [11]:
t = end_time - start_time

In [12]:
# Мы понимаем, что ко времени работы модели добавляем время обработки батчей и добавления аугментаций к стартовым данным

print("Наши метрики на базовой модели")
print(f"Время работы модели на всем батче {t:.<2g} сек.")
print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
print("Метрики качества")
for m in avg_metric:
    print(f"metric {m} : {avg_metric[m]:.<5g}")

Наши метрики на базовой модели
Время работы модели на всем батче 201.197 сек.
Время работы модели на одном сэмпле (AVG) 0.065 сек.
Метрики качества
metric val_RMAP : 0.670246
metric CMAP_5 : 0.762456


In [13]:
# Проверем производительность нашей модели, если мы будем учитывать все веса нашей модели только до тысячных.
params = list(model.parameters()) 
print('the length of parameters is', len(params))
for i in range(len(params)):
    params[i].data = torch.round(params[i].data * 10 ** 3) / 10 ** 3
    params = list(model.parameters())

the length of parameters is 213


In [14]:
# %%timeit -n 4
## Найдем скорость работы базовой модели на нашем валидационном датасете
predicted_labels_list = None
true_labels_list = None
metric = validation_epoch_end
start_time = time.time()

model = model_base
with torch.no_grad():
    for batch in tqdm(valid_loader):
        X_batch = batch[0].to(device)
        y_batch = batch[1].to(device)
        res = model_base.forward(X_batch)

        res = res.detach().cpu().sigmoid().numpy()
        y_batch_onehot = y_batch
        y_batch_onehot = y_batch_onehot.unsqueeze(1).detach().cpu().numpy()
        y_batch_onehot = y_batch_onehot.squeeze()

        if predicted_labels_list is None:
            predicted_labels_list = res
            true_labels_list = y_batch_onehot
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)      

        del batch, res
    torch.cuda.empty_cache()

end_time = time.time()

all_predicted_labels = np.vstack(predicted_labels_list)
all_true_labels = np.vstack(true_labels_list)
all_true_labels = np.squeeze(all_true_labels)
mask = (all_true_labels > 0) & (all_true_labels < 1)
all_true_labels[mask] = 0
avg_metric = metric(all_true_labels, all_predicted_labels)

t = end_time - start_time

# Мы понимаем, что ко времени работы модели добавляем время обработки батчей и добавления аугментаций к стартовым данным

print("Наши метрики на базовой модели, с весами округленными до тысячных ")
print(f"Время работы модели на всем батче {t:.<2g} сек.")
print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
print("Метрики качества")
for m in avg_metric:
    print(f"metric {m} : {avg_metric[m]:.<5g}")

100%|██████████| 771/771 [03:16<00:00,  3.92it/s]


Наши метрики на базовой модели, с весами округленными до тысячных 
Время работы модели на всем батче 196.93 сек.
Время работы модели на одном сэмпле (AVG) 0.064 сек.
Метрики качества
metric val_RMAP : 0.667969
metric CMAP_5 : 0.759319
